In [1]:
import numpy as np
from urllib.request import urlopen
import urllib
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn as skl
import pandas as pd

In [2]:
Cleveland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
np.set_printoptions(threshold=np.nan)
names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heartdisease']
heartDisease = pd.read_csv(urlopen(Cleveland_data_URL), names = names)
heartDisease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heartdisease
0,28,1,2,130,132,0,2,185,0,0.0,?,?,?,0
1,29,1,2,120,243,0,0,160,0,0.0,?,?,?,0
2,29,1,2,140,?,0,0,170,0,0.0,?,?,?,0
3,30,0,1,170,237,0,1,170,0,0.0,?,?,6,0
4,31,0,2,100,219,0,1,150,0,0.0,?,?,?,0


In [3]:
del heartDisease['ca']
del heartDisease['slope']
del heartDisease['thal']
del heartDisease['oldpeak']

heartDisease = heartDisease.replace('?', np.nan)
heartDisease.dtypes

age              int64
sex              int64
cp               int64
trestbps        object
chol            object
fbs             object
restecg         object
thalach         object
exang           object
heartdisease     int64
dtype: object

In [4]:
heartDisease.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'heartdisease'],
      dtype='object')

In [5]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

model = BayesianModel([('age', 'trestbps'), ('age', 'fbs'), ('sex', 'trestbps'), ('sex', 'trestbps'), 
                       ('exang', 'trestbps'),('trestbps','heartdisease'),('fbs','heartdisease'),
                      ('heartdisease','restecg'),('heartdisease','thalach'),('heartdisease','chol')])


model.fit(heartDisease, estimator=MaximumLikelihoodEstimator)
print(model.get_cpds('age'))

╒═════════╤════════════╕
│ age(28) │ 0.00383142 │
├─────────┼────────────┤
│ age(29) │ 0.00383142 │
├─────────┼────────────┤
│ age(30) │ 0.00383142 │
├─────────┼────────────┤
│ age(31) │ 0.00766284 │
├─────────┼────────────┤
│ age(32) │ 0.0153257  │
├─────────┼────────────┤
│ age(33) │ 0.00766284 │
├─────────┼────────────┤
│ age(34) │ 0.0153257  │
├─────────┼────────────┤
│ age(35) │ 0.0191571  │
├─────────┼────────────┤
│ age(36) │ 0.0191571  │
├─────────┼────────────┤
│ age(37) │ 0.0306513  │
├─────────┼────────────┤
│ age(38) │ 0.0191571  │
├─────────┼────────────┤
│ age(39) │ 0.0344828  │
├─────────┼────────────┤
│ age(40) │ 0.0191571  │
├─────────┼────────────┤
│ age(41) │ 0.0383142  │
├─────────┼────────────┤
│ age(42) │ 0.0268199  │
├─────────┼────────────┤
│ age(43) │ 0.0421456  │
├─────────┼────────────┤
│ age(44) │ 0.0268199  │
├─────────┼────────────┤
│ age(45) │ 0.0229885  │
├─────────┼────────────┤
│ age(46) │ 0.045977   │
├─────────┼────────────┤
│ age(47) │ 0.0344828  │


In [6]:
print(model.get_cpds('chol'))

╒══════════════╤══════════════════════╤══════════════════════╕
│ heartdisease │ heartdisease(0)      │ heartdisease(1)      │
├──────────────┼──────────────────────┼──────────────────────┤
│ chol(100)    │ 0.006535947712418301 │ 0.006535947712418301 │
├──────────────┼──────────────────────┼──────────────────────┤
│ chol(117)    │ 0.006535947712418301 │ 0.006535947712418301 │
├──────────────┼──────────────────────┼──────────────────────┤
│ chol(129)    │ 0.006535947712418301 │ 0.006535947712418301 │
├──────────────┼──────────────────────┼──────────────────────┤
│ chol(132)    │ 0.006535947712418301 │ 0.006535947712418301 │
├──────────────┼──────────────────────┼──────────────────────┤
│ chol(147)    │ 0.006535947712418301 │ 0.006535947712418301 │
├──────────────┼──────────────────────┼──────────────────────┤
│ chol(156)    │ 0.006535947712418301 │ 0.006535947712418301 │
├──────────────┼──────────────────────┼──────────────────────┤
│ chol(160)    │ 0.006535947712418301 │ 0.0065359477124

In [7]:
print(model.get_cpds('sex'))

╒════════╤══════════╕
│ sex(0) │ 0.264368 │
├────────┼──────────┤
│ sex(1) │ 0.735632 │
╘════════╧══════════╛


In [8]:
model.get_independencies()

(age _|_ exang, sex)
(age _|_ sex | exang)
(age _|_ exang | sex)
(age _|_ thalach, chol, restecg | heartdisease)
(age _|_ exang, sex | fbs)
(age _|_ thalach, chol, restecg | exang, heartdisease)
(age _|_ sex | exang, fbs)
(age _|_ thalach, chol, restecg | trestbps, heartdisease)
(age _|_ thalach, chol, heartdisease, restecg | trestbps, fbs)
(age _|_ thalach, chol, restecg | sex, heartdisease)
(age _|_ exang | sex, fbs)
(age _|_ thalach, restecg | chol, heartdisease)
(age _|_ chol, restecg | thalach, heartdisease)
(age _|_ thalach, chol, restecg | heartdisease, fbs)
(age _|_ thalach, chol | heartdisease, restecg)
(age _|_ thalach, chol, restecg | exang, trestbps, heartdisease)
(age _|_ thalach, chol, heartdisease, restecg | exang, trestbps, fbs)
(age _|_ thalach, chol, restecg | exang, sex, heartdisease)
(age _|_ thalach, restecg | exang, chol, heartdisease)
(age _|_ chol, restecg | exang, thalach, heartdisease)
(age _|_ thalach, chol, restecg | exang, heartdisease, fbs)
(age _|_ thalac

In [9]:
from pgmpy.inference import VariableElimination
HeartDisease_infer = VariableElimination(model)

q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'age': 28})
print(q['heartdisease'])

╒════════════════╤═════════════════════╕
│ heartdisease   │   phi(heartdisease) │
╞════════════════╪═════════════════════╡
│ heartdisease_0 │              0.5770 │
├────────────────┼─────────────────────┤
│ heartdisease_1 │              0.4230 │
╘════════════════╧═════════════════════╛


c:\users\harshith\appdata\local\programs\python\python37\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
c:\users\harshith\appdata\local\programs\python\python37\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


In [10]:
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'chol': 100})
print(q['heartdisease'])

╒════════════════╤═════════════════════╕
│ heartdisease   │   phi(heartdisease) │
╞════════════════╪═════════════════════╡
│ heartdisease_0 │              0.5510 │
├────────────────┼─────────────────────┤
│ heartdisease_1 │              0.4490 │
╘════════════════╧═════════════════════╛
